In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 43.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorrt==8.6.1

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=ed2d19f8251c444b1fe305ae2720ab65c4b0078643493cb723df49702878a56e
  Stored in directory: /root/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt


In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=c9ab1136b3ce58b27550f6baa33d05b220e91a4a617deb987d5705e37c8d26bd
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np

# 加载预训练的PyTorch模型
model = models.mobilenet_v2(pretrained=True)
model.eval()

# 定义图像预处理流程
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 加载并预处理图像
img = Image.open('cat.jpg')
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0).numpy()

# 导出模型到ONNX格式
torch.onnx.export(model,
                  torch.from_numpy(input_batch),
                  "mobilenetv2.onnx",
                  verbose=True,
                  export_params=True,
                  opset_version=11)

# 创建TensorRT构建器
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)

# 使用新的网络定义方式（适用于TensorRT 9.x）
network_flags = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
network = builder.create_network(network_flags)
parser = trt.OnnxParser(network, TRT_LOGGER)

# 解析ONNX模型
with open("mobilenetv2.onnx", "rb") as model_file:
    if not parser.parse(model_file.read()):
        for error in range(parser.num_errors):
            print(parser.get_error(error))
        raise RuntimeError("Failed to parse ONNX file.")

# 配置构建器
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1GB

# 启用FP16模式（如果支持）
if builder.platform_has_fast_fp16:
    config.set_flag(trt.BuilderFlag.FP16)

# 构建TensorRT引擎
engine = builder.build_engine(network, config)
if engine is None:
    raise RuntimeError("Failed to build the engine.")

# 序列化并保存引擎
with open("mobilenetv2.trt", "wb") as f:
    f.write(engine.serialize())

# 加载TensorRT引擎
runtime = trt.Runtime(TRT_LOGGER)
with open("mobilenetv2.trt", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

# 创建执行上下文
context = engine.create_execution_context()

# 分配内存
inputs, outputs, bindings, stream = [], [], [], cuda.Stream()
for binding in engine:
    binding_idx = engine.get_binding_index(binding)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    dtype = trt.nptype(engine.get_binding_dtype(binding))
    host_mem = cuda.pagelocked_empty(size, dtype)
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    bindings.append(int(device_mem))
    if engine.binding_is_input(binding):
        inputs.append({'host': host_mem, 'device': device_mem})
    else:
        outputs.append({'host': host_mem, 'device': device_mem})

# 准备输入数据
inputs[0]['host'] = input_batch.ravel()

# 将输入数据复制到设备
cuda.memcpy_htod_async(inputs[0]['device'], inputs[0]['host'], stream)

# 执行推理
context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)

# 将输出数据复制回主机
cuda.memcpy_dtoh_async(outputs[0]['host'], outputs[0]['device'], stream)

# 等待推理完成
stream.synchronize()

# 处理输出
output_data = np.array(outputs[0]['host']).reshape(1, -1)
predicted_class = np.argmax(output_data, axis=1)

print('预测类别索引:', predicted_class[0])

<ipython-input-8-288cacf256a2>:56: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1GB
<ipython-input-8-288cacf256a2>:63: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


预测类别索引: 285


<ipython-input-8-288cacf256a2>:82: DeprecationWarning: Use get_tensor_name instead.
  binding_idx = engine.get_binding_index(binding)
<ipython-input-8-288cacf256a2>:83: DeprecationWarning: Use get_tensor_shape instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
<ipython-input-8-288cacf256a2>:83: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
<ipython-input-8-288cacf256a2>:84: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(engine.get_binding_dtype(binding))
<ipython-input-8-288cacf256a2>:88: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):


In [ ]:
!nvidia-smi

Thu Oct 31 08:39:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--